In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
import dagshub
dagshub.init(repo_owner='MolAlexandre', repo_name='Chest_Cancer', mlflow=True)

import mlflow

Accessing as MolAlexandre

Initialized MLflow to track repo "MolAlexandre/Chest_Cancer"

Repository MolAlexandre/Chest_Cancer initialized!

In [4]:
import tensorflow as tf

In [5]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [7]:
from Chest_Cancer_Classifier.constants import *
from Chest_Cancer_Classifier.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Data",
            mlflow_uri="https://dagshub.com/MolAlexandre/Chest_Cancer.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from mlflow.models import infer_signature

from keras.preprocessing.image import ImageDataGenerator

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        valid_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)
        
        self.val_dataset  = valid_datagen.flow_from_directory(
                    directory = os.path.join(self.config.training_data,'valid'),
                    target_size = self.config.params_image_size[:-1],
                    class_mode = 'categorical',
                    batch_size = self.config.params_batch_size
                    )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.val_dataset)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            # Set a tag that we can use to remind ourselves what this run was for
            mlflow.set_tag("Training Info", "Basic LR model for iris data")

            signature = infer_signature(self.config.params_image_size[:-1], tf.TensorShape(0,4))
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model",signature=signature)
            else:
                mlflow.keras.log_model(self.model, "model")

In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-08-28 22:17:34,799: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-28 22:17:34,802: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-28 22:17:34,803: INFO: common: created directory at: artifacts]
Found 72 images belonging to 4 classes.
5/5 [==============================] - 8s 2s/step - loss: 28.0527 - accuracy: 0.2917
[2024-08-28 22:17:43,194: INFO: common: json file saved at: scores.json]


TypeError: TensorShape.__init__() takes 2 positional arguments but 3 were given